In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
from docx import Document
import pdfplumber
from bs4 import BeautifulSoup
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
spacy.cli.download("en_core_web_sm")
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
from plotly.offline import plot
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nicholasreese/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
#!pip install transformers torch

In [3]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline

/Users/nicholasreese/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning:

`torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.



***

## Loading the Model and Tokenizer

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
resume_text = """

Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese15@gmail.com 
Education 
•Georgetown University - August 2023 – December 2024
Master of Science Business Analytics – MSBA candidate
Peer-Elected Class Representative 
•Dickinson College - August 2014 – September 2018
Economics & Political Science Double Major
Varsity Tennis Captain 
•University of Bologna - August 2016

Skills
- R studio
- Python
- Power BI
- SQL
- Neural Networks
- Machine Learning
- Macro Modeling
- AWS Cloud Services
- Pandas
- A/B Testing
- Scikit-Learn
- Econometrics

Professional Experience
FINRA, Washington, D.C.	June 2020 – Present 
Senior Analyst, Market Regulation	September 2022 – Present 
•Earned top 10% of performance of analysts for past two years.
•Led team in research implementing PostgreSQL and NoSQL queries for large data pulls.
•Spearheaded new analytical approaches to financial workflow with tools such as R & Python for model development.
•Developed working predictive modeling schemas for senior staff using statistical analysis.
•Leveraged skills in platforms like Python, R, Power BI, Tableau and SQL accompanied with strong statistical background in financial markets.
•Built the Security-Based Swap training manual deck & produced the recorded info session for all of FINRA. 
•Selected for the FINRA’s first Georgetown Advanced Analytics Program as one of the most junior staff awarded opportunity.
•Incorporated statistical analytics to assist in creating a NPL tool to analyze financial documents language to minimize time on manual analysis.
•Produced unsupervised and supervised models to perform analysis for Security-Based Swaps trade patterns.
•Improved FINRA platforms with Data Scientists for higher accuracy and efficiency. 
•Managed process of re-engineering supervisory reviews through advanced analytic tools for senior staff.
•Implements advanced analytics to assist senior staff with maximizing efficiencies in daily workloads and trade pattern creations.
•Presented visualization case work using Power BI & Tableau to senior leadership.
•Persuaded senior staff to allow for statistical analytics tools like R.
•Mentored over 20 junior & senior staff members on improving processes with analytical tools for financial reviews.
•Managed junior staff with day-to-day workload while teaching the staff how to use advanced analytics.
Analyst, Market Regulation	June 2021-September 2022
·Created macroeconomic models for newly implemented FINRA Rule 2232 and MSRB Rule G-15.
·Mentored junior staff with creating macro models, synthesizing responses from FINRA member firms.
·Won the Regulator Scholarship for continued financial learning based on individual performance.
·Received six internal awards for expertise in Municipal and Corporate Bond analysis as an analyst.
Associate Analyst, Market Regulation	June 2020 – June 2021
·Created macroeconomic models to quantify business models of selected firms for FINRA Rule 2232.
             BNY Mellon, Pittsburgh, PA						                 September 2019 – June 2020
Corporate Trust Associate
·Leader & instructor of the Bloomberg software for the Corporate Trust Team.
LendingHome, Internship, Pittsburgh, PA
Funding Specialist & Post Closing Member	March 2019 – August 2019
·Reviewed closing documents to ensure precise execution for funding staff.
              Veraction/Trax, Junior Business Analyst, Memphis, TN	  June 2016 - August 2016                                                    June 2016 - August 2016
·Led the creation, development, and implementation of Request For Proposal (RFP) database.
"""


In [6]:
#resume_text

***

In [7]:
from transformers import pipeline

In [8]:
ner_pipeline = pipeline('ner', model = "dbmdz/bert-large-cased-finetuned-conll03-english",
                       aggregation_strategy = 'simple')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
entities = ner_pipeline(resume_text)

In [10]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Science, Label: MISC, Score: 0.7188
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##tics, Label: MISC, Score: 0.6904
Entity: MS, Label: MISC, Score: 0.6772
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: Science, Label: MISC, Score: 0.6226
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: Python, Label: MISC, Score: 0.7331
Entity: B, Label: MISC, Score: 0.3749
Entity: S, Label: MISC, Score: 0.4090
Entity: N, Label: MISC, Score: 0.3621
Entity: Networks, Label: MISC, Score:

In [11]:
filtered_entities = [entity for entity in entities if entity['entity_group'] != 'MISC']

In [12]:
for entity in filtered_entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: AWS, Label: ORG, Score: 0.8358
Entity: FINRA, Label: ORG, Score: 0.9555
Entity: Washington, Label: LOC, Score: 0.9981
Entity: D, Label: LOC, Score: 0.9980
Entity: C, Label: LOC, Score: 0.9992
Entity: Regulation, Label: ORG, Score: 0.4722
Entity: Power, Label: ORG, Score: 0.4565
Entity: Table, Label: ORG, Score: 0.5572
Entity: FINRA, Label: ORG, Score: 0.

In [13]:
import re

# Regex pattern for email extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

# Find all email addresses in the text
emails = re.findall(email_pattern, resume_text)

# Display found email addresses
print(emails)

['Nicholas.E.Reese15@gmail.com']


In [14]:
# Load the NER pipeline
ner_pipeline2 = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")


# Detect entities using the NER pipeline
entities2 = ner_pipeline2(resume_text)

# Regex pattern for email extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

# Find all email addresses in the text
emails = re.findall(email_pattern, resume_text)

# Print detected entities from NER
for entity in entities2:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")



Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Science, Label: MISC, Score: 0.7188
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##tics, Label: MISC, Score: 0.6904
Entity: MS, Label: MISC, Score: 0.6772
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: Science, Label: MISC, Score: 0.6226
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: Python, Label: MISC, Score: 0.7331
Entity: B, Label: MISC, Score: 0.3749
Entity: S, Label: MISC, Score: 0.4090
Entity: N, Label: MISC, Score: 0.3621
Entity: Networks, Label: MISC, Score: